In [ ]:
from ocpy import OCP
from ocpy import DDPSolver, iLQRSolver
from ocpy import symutils

import numpy as np
import sympy as sym
from sympy import sin, cos, tan, exp, log, ln, sinh, cosh, tanh, diff, sqrt


In [ ]:
# Dimensions of state and input
n_x = 4
n_u = 1
sim_name = 'cartpole'

# Define ocp class
ocp = OCP(sim_name, n_x, n_u)

# Get symbols
t = ocp.get_t()
x = ocp.get_x()
u = ocp.get_u()


In [ ]:
# Symbolic expressions of constants.
m_c, m_p, l, g_c, u_min, u_max = ocp.define_scalar_constants(
         [('m_c', 2), ('m_p', 0.2), ('l', 0.5), ('g', 9.80665), 
          ('u_min', -15),  ('u_max', 15)]
)

# Cost weight
q = ocp.define_vector_constant('q', [2.5, 10, 0.01, 0.01])
r = ocp.define_vector_constant('r', [1])
q_f = ocp.define_vector_constant('q_{f}', [2.5, 10, 0.01, 0.01])
Q = symutils.diag(q)
Q_f = symutils.diag(q_f)
R = symutils.diag(r)

# Reference state. 
x_ref = ocp.define_vector_constant('x_{ref}', [0, np.pi, 0, 0])


In [ ]:
# State space representation.
f = ocp.get_zero_vector(n_x)
f[0] = x[2]
f[1] = x[3]
f[2] = (u[0] + m_p*sin(x[1])*(l*x[3]*x[3] + g_c*cos(x[1])) )/( m_c+m_p*sin(x[1])*sin(x[1]) )
f[3] = (-u[0] * cos(x[1]) - m_p*l*x[3]*x[3]*cos(x[1])*sin(x[1]) 
        - (m_c+m_p)*g_c*sin(x[1]) )/( l*(m_c + m_p*sin(x[1])*sin(x[1])))

# Log barrier / exp penalty for control limits.
# u_penalty = ocp.get_penalty(u, [u_min], [u_max], mu=1e0)
u_barrier = ocp.get_barrier(u, [u_min], [u_max])

# Stage cost and terminal cost.
l = 0.5 * (x - x_ref).T * Q * (x - x_ref) + 0.5 * u.T * R * u + u_barrier
lf = 0.5 * (x - x_ref).T * Q_f * (x - x_ref)


In [ ]:
# Horizon length and discretization grids.
T = 5.0
N = 200

# Initial condition
t0 = 0.0
x0 = np.array([0.0, 0.0, 0.0, 0.0])

# Define ocp
ocp.define_unconstrained(f, l, lf, t0, x0, T, N)


In [ ]:
# Display problem
ocp.display()

In [ ]:
# Hand over ocp to solver.
# DDP, based on exact Hessian.
solver = DDPSolver(ocp)

# iLQR, Based on Gauss-Newton-like approximated Hessian.
# solver = iLQRSolver(ocp)


In [ ]:
# If needed, reset parameters and set solution guess.
n_x = ocp.get_n_x()
n_u = ocp.get_n_u()
T = ocp.get_T()
N = ocp.get_N()
t0 = ocp.get_t0()
x0 = ocp.get_x0()

# t0 = 0.0
# x0 = np.zeros(n_x)
# solver.set_initial_condition(t0=t0, x0=x0)

# T = 5.0
# N = 200
# solver.set_horizon(T=T, N=N)

# solver.reset_guess()
us_guess = np.zeros((N, n_u))
solver.set_guess(us_guess=us_guess)


In [ ]:
# set parameters.
solver.set_line_search_param(alpha_min=1e-3, r_alpha=0.5)
# solver.set_regularization_param(
#     gamma_init=1e-3, r_gamma=5.0, gamma_min=0.0, gamma_max=1e6
# )
solver.set_stop_tol(1e-3)
solver.set_max_iters(1000)

# Solve ocp
solver.solve(
    update_gamma=False, enable_line_search=True,
    result=True, log=True, plot=True
)


In [ ]:
%matplotlib inline
# Visualize
from ocpy.animator import CartPoleAnimator
animator = CartPoleAnimator(solver.get_log_directory(), sim_name)
animator.generate_animation(False)


In [ ]:
import matplotlib.pyplot as plt
import math

result = solver.get_result()

gamma_hist = result['gamma_hist']
plt.plot(gamma_hist)
plt.title('gamma')
plt.show()

alpha_hist = result['alpha_hist']
plt.plot(alpha_hist)
plt.title('alpha')
plt.show()
print('average alpha:',sum(alpha_hist / (len(alpha_hist) - 1)))

cost_hist = result['cost_hist']
plt.plot(cost_hist)
plt.title('cost')
plt.show()
